In [1]:
%%capture
%pip install torch
%pip install torchvision


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [3]:
# Load the heart disease dataset
data = pd.read_csv('heart_disease_dataset.csv')
print(data.head(10))

# Split features and target variable
X = data.drop(columns=['target'])
y = data['target']

# Convert dataframe to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_tensor)

# Training split 1 is 80/10/10
# Training split 2 is 70/15/15
# Training split 3 is 60/20/20

split_num = input("Training split 1, 2, or 3: ")

if split_num == "1":
  X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_tensor, test_size=0.2, random_state=42)
  X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
elif split_num == "2":
  X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_tensor, test_size=0.3, random_state=42)
  X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
elif split_num == "3":
  X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_tensor, test_size=0.4, random_state=42)
  X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)




   age  sex  chest pain type  resting bp s  cholesterol  fasting blood sugar  \
0   40    1                2           140          289                    0   
1   49    0                3           160          180                    0   
2   37    1                2           130          283                    0   
3   48    0                4           138          214                    0   
4   54    1                3           150          195                    0   
5   39    1                3           120          339                    0   
6   45    0                2           130          237                    0   
7   54    1                2           110          208                    0   
8   37    1                4           140          207                    0   
9   48    0                2           120          284                    0   

   resting ecg  max heart rate  exercise angina  oldpeak  ST slope  target  
0            0             172            

In [ ]:
# Define custom dataset for PyTorch
class HeartDiseaseDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Create dataset and dataloaders
train_dataset = HeartDiseaseDataset(X_train, y_train)
valid_dataset = HeartDiseaseDataset(X_valid, y_valid)
test_dataset = HeartDiseaseDataset(X_test, y_test)

# Batch size of 1 means each change is based of off a single training example
# Batch size of 50 is medium size with a stable convergence
# Batch size of 250+ is slow convergence and bad generalization but efficient
batch_size = int(input("Define the batch size: "))

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)


Define the batch size: 50


In [ ]:
# Define the first model
class HeartDiseaseModel1(nn.Module):
    def __init__(self):
        super(HeartDiseaseModel1, self).__init__()
        input_size = X_train.shape[1]

        #This dymanic first layer is fully connected as it takes input_size neurons
        self.fc1 = nn.Linear(input_size, 400)

        # ReLU refers to Rectified Linear Unit and is a common activation function
        # its a piecewise linear function that outputs the input directly if positive
        # if the input is negative, it outputs 0
        self.relu = nn.ReLU()

        self.fc2 = nn.Linear(400, 1)

        #Final layer applies the sigmoid function to force the output between 0 and 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

#Define the second model
# this model has one less layer and far less neurons compared to the other models
class HeartDiseaseModel2(nn.Module):
    def __init__(self):
        super(HeartDiseaseModel2, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)


    def forward(self, x):
        x = F.relu(self.fc1(x))  # Activation function for first layer
        x = F.relu(self.fc2(x))  # Activation function for second layer
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for output layer
        return x

#Define the third model
# This model is exceptionally similar to the first one but with an extra layer
class HeartDiseaseModel3(nn.Module):
    def __init__(self):
        super(HeartDiseaseModel3, self).__init__()
        input_size = X_train.shape[1]
        # This dynamic first layer is fully connected as it takes input_size neurons
        self.fc1 = nn.Linear(input_size, 400)

        # Adding an additional fully connected layer
        self.fc2 = nn.Linear(400, 50)  # New layer with 50 neurons

        # same as teh original
        self.relu = nn.ReLU()

        self.fc3 = nn.Linear(50, 1)

        # Final layer applies the sigmoid function to force the output between 0 and 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
num = input("Test 1, 2, or 3: ")
# Initialize the model and move it to the GPU if available
if num == "1":
  model = HeartDiseaseModel1()
elif num == "2":
  model = HeartDiseaseModel2()
elif num == "3":
  model = HeartDiseaseModel3()

if torch.cuda.is_available():
    model.cuda()
LR = input("Learning rate 1, 2, 3, or 4: ")
if LR == "1":
  LEARNING_RATE = 0.01 #the model will learn very quickly
elif LR == "2":
  LEARNING_RATE = 0.001 # not too slow but not too fast
elif LR == "3":
  LEARNING_RATE = 0.0001 # good for gradual changes, unlikely to overshoot loss function
elif LR == "4":
  LEARNING_RATE = 0.00001 # incredibly slow, very fine tuning

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Test 1, 2, or 3: 3
Learning rate 1, 2, 3, or 4: 3


In [ ]:
#Epochs are the total number of times the algorithm goes through the whole set
# 5-10 epochs will indicate underfitting and how much can be learned quickly
# 50 epochs will sufficiently catch complex patterns without overfitting
# 200+ will point out overfitting
NUM_EPOCHS = int(input("How many epochs: "))
losses = []
for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for i, (X_batch, y_batch) in enumerate(train_loader):
        if torch.cuda.is_available():
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

        # Convert input data to float32
        X_batch = X_batch.float()

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_function(outputs, y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print(f'Step {i+1} / {len(train_loader)}, Loss: {loss.item()}')

How many epochs: 50
Epoch 1/50
Step 1 / 20, Loss: 0.6986067295074463
Step 2 / 20, Loss: 0.6906841397285461
Step 3 / 20, Loss: 0.6908621788024902
Step 4 / 20, Loss: 0.6818970441818237
Step 5 / 20, Loss: 0.6911101341247559
Step 6 / 20, Loss: 0.6770676374435425
Step 7 / 20, Loss: 0.6756614446640015
Step 8 / 20, Loss: 0.6632146239280701
Step 9 / 20, Loss: 0.6759188175201416
Step 10 / 20, Loss: 0.6555429697036743
Step 11 / 20, Loss: 0.6740960478782654
Step 12 / 20, Loss: 0.6590105295181274
Step 13 / 20, Loss: 0.6565342545509338
Step 14 / 20, Loss: 0.6671660542488098
Step 15 / 20, Loss: 0.6605589389801025
Step 16 / 20, Loss: 0.6643780469894409
Step 17 / 20, Loss: 0.6549785733222961
Step 18 / 20, Loss: 0.660632312297821
Step 19 / 20, Loss: 0.6423093676567078
Step 20 / 20, Loss: 0.7053725719451904
Epoch 2/50
Step 1 / 20, Loss: 0.6418484449386597
Step 2 / 20, Loss: 0.6442711353302002
Step 3 / 20, Loss: 0.6399983167648315
Step 4 / 20, Loss: 0.6288543343544006
Step 5 / 20, Loss: 0.624051570892334

In [ ]:
# Define function to get accuracy
def get_accuracy(data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            if torch.cuda.is_available():
                X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

            # Convert input data to float32
            X_batch = X_batch.float()

            outputs = model(X_batch)
            predicted = torch.round(outputs)
            correct += (predicted == y_batch.unsqueeze(1)).sum().item()
            total += y_batch.size(0)

    return (correct / total) * 100

# Calculate accuracy on train, validation, and test sets
train_accuracy = get_accuracy(train_loader)
valid_accuracy = get_accuracy(valid_loader)
test_accuracy = get_accuracy(test_loader)

print("Train accuracy: ", train_accuracy)
print("Valid accuracy: ", valid_accuracy)
print("Test accuracy: ", test_accuracy)


Train accuracy:  85.92436974789915
Valid accuracy:  85.71428571428571
Test accuracy:  89.91596638655463
